In [13]:
path_xlsx_parte = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [14]:
import sys
sys.path.append('../_amigocloud')

In [15]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [16]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [17]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [24]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [25]:
df_lib = pd.DataFrame(data)

In [26]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,189,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
1,190,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
2,191,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
3,193,52,2024-07-15 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 15:11:19+00:00,2024-07-16 21:07:19+00:00,LIBERACION,20,15,65,CONCLUIDO,MT 02,None
4,200,51,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 13:07:02+00:00,2024-07-16 16:12:02+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
5,203,51,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 13:07:02+00:00,2024-07-16 16:12:02+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
6,208,50,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 18:07:27+00:00,2024-07-16 22:05:27+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
7,209,53,2024-07-10 11:30:14+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-05 14:30:14+00:00,2024-07-05 19:30:14+00:00,LIBERACION,17,20,65,CONCLUIDO,MT 01,None
8,81,47,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 15:05:38+00:00,2024-07-17 19:43:38+00:00,LIBERACION,25,10,60,CONCLUIDO,MT 01,None
9,87,48,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 12:48:39+00:00,2024-07-16 14:52:37+00:00,LIBERACION,25,10,60,CONCLUIDO,MT 01,None


In [27]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[46, 47, 48, 49, 50, 51, 52, 53]

In [28]:
lista_id = list(set(df_lib['id']))
lista_id

[206,
 207,
 168,
 86,
 189,
 190,
 191,
 192,
 193,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 72,
 203,
 204,
 202,
 205,
 73,
 208,
 209,
 81,
 210,
 211,
 82,
 212,
 87,
 88,
 213,
 83,
 89,
 84,
 90,
 85,
 91,
 92,
 99,
 74,
 75,
 76]

In [29]:
nulos = df_lib.isnull().sum()
nulos

id                      0
idd                     0
fecha_registro          0
piloto_1                0
piloto_2                0
hora_inicio             0
hora_fin                0
labor                   0
temperatura             0
viento                  0
humedad                 0
estado_de_operacion     0
dron                    0
obs                    43
dtype: int64

In [30]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [31]:
if null_check:
    df_lib = None
else:
    None

In [32]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,189,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
1,190,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
2,191,49,2024-07-16 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 13:32:43+00:00,2024-07-16 18:10:43+00:00,LIBERACION,19,10,60,CONCLUIDO,MT 02,None
3,193,52,2024-07-15 19:29:47+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-16 15:11:19+00:00,2024-07-16 21:07:19+00:00,LIBERACION,20,15,65,CONCLUIDO,MT 02,None
4,200,51,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 13:07:02+00:00,2024-07-16 16:12:02+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
5,203,51,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 13:07:02+00:00,2024-07-16 16:12:02+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
6,208,50,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 18:07:27+00:00,2024-07-16 22:05:27+00:00,LIBERACION,20,15,60,CONCLUIDO,MT 01,None
7,209,53,2024-07-10 11:30:14+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-05 14:30:14+00:00,2024-07-05 19:30:14+00:00,LIBERACION,17,20,65,CONCLUIDO,MT 01,None
8,81,47,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 15:05:38+00:00,2024-07-17 19:43:38+00:00,LIBERACION,25,10,60,CONCLUIDO,MT 01,None
9,87,48,2024-07-16 19:29:47+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-16 12:48:39+00:00,2024-07-16 14:52:37+00:00,LIBERACION,25,10,60,CONCLUIDO,MT 01,None


In [33]:
df_parte = pd.read_excel(path_xlsx_parte)

In [34]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE
0,641,2024-05-23,21.0,8,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERAZANZA - SUAREZ DAVID,L1,17.000000,...,DA 02,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
1,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L9.2,17.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
2,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L13,10.710000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
3,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C10.3,2.490000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
4,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C12.2,0.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,NaN,NaT,NaN,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C14.3,1.066949,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,209.0,33.0,NaN,NaN
238,NaN,NaT,NaN,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C14.2,0.450754,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,210.0,33.0,NaN,NaN
239,NaN,NaT,NaN,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C14.1,1.128854,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,211.0,33.0,NaN,NaN
240,NaN,NaT,NaN,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C13.1,0.995924,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,212.0,33.0,NaN,NaN


In [35]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_13896\1834054455.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
C:\Users\bismarksr\AppData\Local\Temp\ipykernel_13896\1834054455.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')


In [36]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_13896\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [37]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [38]:
lista_idd

[46, 47, 48, 49, 50, 51, 52, 53]

In [39]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 46
se proceso IDD: 47
se proceso IDD: 48
se proceso IDD: 49
se proceso IDD: 50
se proceso IDD: 51
se proceso IDD: 52
se proceso IDD: 53


In [40]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 206
se proceso ID: 207
se proceso ID: 168
se proceso ID: 86
se proceso ID: 189
se proceso ID: 190
se proceso ID: 191
se proceso ID: 192
se proceso ID: 193
se proceso ID: 195
se proceso ID: 196
se proceso ID: 197
se proceso ID: 198
se proceso ID: 199
se proceso ID: 200
se proceso ID: 201
se proceso ID: 72
se proceso ID: 203
se proceso ID: 204
se proceso ID: 202
se proceso ID: 205
se proceso ID: 73
se proceso ID: 208
se proceso ID: 209
se proceso ID: 81
se proceso ID: 210
se proceso ID: 211
se proceso ID: 82
se proceso ID: 212
se proceso ID: 87
se proceso ID: 88
se proceso ID: 213
se proceso ID: 83
se proceso ID: 89
se proceso ID: 84
se proceso ID: 90
se proceso ID: 85
se proceso ID: 91
se proceso ID: 92
se proceso ID: 99
se proceso ID: 74
se proceso ID: 75
se proceso ID: 76
